In [7]:
import numpy as np

# Reduce displayed waveforms to avoid display freezes
CSI_VAID_SUBCARRIER_INTERVAL = 1

csi_vaid_subcarrier_index = []
csi_vaid_subcarrier_color = []
color_step = 255 // (28 // CSI_VAID_SUBCARRIER_INTERVAL + 1)
color_step_57 = 255 // (57 // CSI_VAID_SUBCARRIER_INTERVAL + 1)

csi_vaid_subcarrier_index += [i for i in range(0, 28, CSI_VAID_SUBCARRIER_INTERVAL)]    
csi_vaid_subcarrier_color += [(i * color_step, 0, 0) for i in range(1,  28 // CSI_VAID_SUBCARRIER_INTERVAL + 1)]

csi_vaid_subcarrier_index += [i for i in range(28, 29, CSI_VAID_SUBCARRIER_INTERVAL)]    
csi_vaid_subcarrier_color += [(255, 255, 255) for i in range(1,  1 // CSI_VAID_SUBCARRIER_INTERVAL + 1)]

csi_vaid_subcarrier_index += [i for i in range(29, 57, CSI_VAID_SUBCARRIER_INTERVAL)]   
csi_vaid_subcarrier_color += [(0, i * color_step, 0) for i in range(1,  28 // CSI_VAID_SUBCARRIER_INTERVAL + 1)]
CSI_DATA_114_COLUMNS = len(csi_vaid_subcarrier_index)

csi_vaid_subcarrier_index += [i for i in range(57, 60, CSI_VAID_SUBCARRIER_INTERVAL)]   
csi_vaid_subcarrier_color += [(255, 255, 255)] * 3  

csi_vaid_subcarrier_index += [i for i in range(60, 117, CSI_VAID_SUBCARRIER_INTERVAL)]    
csi_vaid_subcarrier_color += [(0, 0, i * color_step_57) for i in range(1,  57 // CSI_VAID_SUBCARRIER_INTERVAL + 1)]
CSI_DATA_234_COLUMNS = len(csi_vaid_subcarrier_index)

CSI_DATA_COLUMNS = len(csi_vaid_subcarrier_index)
DATA_COLUMNS_NAMES = ["type", "id", "mac", "rssi", "rate","noise_floor","fft_gain","agc_gain", "channel", "local_timestamp",  "sig_len", "rx_state", "len", "first_word", "data"]

In [ ]:
filepath = 'YOUR CSI File Path(.csv)'
import pandas as pd
import os
import ast
from tqdm import tqdm

print('Processing File:',filepath)
df = pd.read_csv(filepath)
csi = df['data']

row = 0
for csi_raw_data in csi:
    csi_raw_data = ast.literal_eval(csi_raw_data)
    if len(csi_raw_data) == 106:
        continue
        csi_vaid_subcarrier_len = CSI_DATA_106_COLUMNS
    elif  len(csi_raw_data) == 114:
        csi_vaid_subcarrier_len = CSI_DATA_114_COLUMNS
    elif  len(csi_raw_data) == 128 :
        continue
        csi_vaid_subcarrier_len = CSI_DATA_LLFT_COLUMNS
    elif  len(csi_raw_data) == 234 :
        csi_vaid_subcarrier_len = CSI_DATA_234_COLUMNS
    else :
        csi_vaid_subcarrier_len = CSI_DATA_COLUMNS
    if row == 0:
        print('Data Sub:',len(csi_raw_data))
        csi_complex_data = np.zeros((len(csi), csi_vaid_subcarrier_len), dtype=complex)
    for i in range(csi_vaid_subcarrier_len):
        data = complex(csi_raw_data[csi_vaid_subcarrier_index[i] * 2 + 1],
                        csi_raw_data[csi_vaid_subcarrier_index[i] * 2])
        csi_complex_data[row,i] = data
    row += 1

print(csi_complex_data)
# CSI shape: (samples, subcarriers)
print(csi_complex_data.shape)

Processing File: ESP/test.csv
Data Sub: 234
[[ -9. +3.j  -8. +4.j -10. +4.j ...   5.+14.j   4.+13.j   6.+14.j]
 [ -6. +3.j  -7. +3.j  -8. +3.j ...  13. -6.j  13. -6.j  12. -7.j]
 [ -6. +0.j  -5. +0.j  -6. +0.j ...   3.-12.j   3.-13.j   1.-12.j]
 ...
 [  1.+11.j   1.+10.j   2.+13.j ...   4. -7.j   2. -7.j   2. -6.j]
 [ 11. +0.j  12. +1.j  11. +0.j ...  -1. -5.j   0. -7.j   0. -7.j]
 [  3.+11.j   4.+11.j   3.+11.j ...   2. +7.j   1. +9.j   2. +7.j]]
(1679, 117)
